# Compute Frag Meth Summaries for a Region Set

Given a region set will compute frag scores for all samples in PAT Parquet files. Here we use hg38 PARQUET files.

SET UP SDDs TO BE USED WITH SPARK PRIOR TO RUNNING THIS NOTEBOOK!


In [1]:
import pandas as pd

import numpy as np
import itertools
import functools
import os
import regex as re

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

## Initialisation

### Parameters

In [27]:
#REGIONS = 'dmr_literature'
#REGIONS = 'loyfer2022_atlas_final_regions_top25'
#REGIONS = 'dmr_tissue'
#REGIONS = 'dmr_lunar_v6'
REGIONS = 'deconvolution_v2.v23_conv.with_cpg_index'
REGION_BED_COLS = [
    'region_chr', 'region_start', 'region_end', 
    'region_cpg_index_min', 'region_cpg_index_max', 'region_id'
]
FILTER_CG_COUNT = 3
FILTER_CG_COUNT_REGION = 1
#--- Local paths
ROOT_DIR = '/analysis/gh-aselewa/projects'
PROJECT_SLUG = '2023_06_15_BCdeconvolution_AS'
PROJECT_DIR = ROOT_DIR + '/{}'.format(PROJECT_SLUG)
# Regions
REGION_PATH = (
    PROJECT_DIR + '/stage/panel_data/{regions}.bed'
).format(regions=REGIONS)

# CpG map; genomic coordinate to CpG index;
CPG_MAP_PATH = PROJECT_DIR + '/stage/cpg_loci/cpg_loci_hg19.combined_annot.tsv.gz'

# # PAT PARQUET files
# PARQUET_PATH_LIST_HG19 = [
#     '/temp/Loyfer_parquet',
# ]

# BLUEPRINT HG38: s3://gh-bi-lunar/public_data/blueprint/hg38_20160816.pat.db_version.parquet/
PARQUET_PATH_LIST_HG38 = [
    '/temp/blueprint_hg38/'
]

#--- Where to store results
RESULTS_PATH = (
    PROJECT_DIR + '/output/blueprint_meth_summaries/meth_summaries_cg_count_geq_{k}_{regions}.tsv.gz'
).format(regions=REGIONS, k=FILTER_CG_COUNT)

### Spark Setup

In [3]:
from pyspark.sql import SparkSession
from pyspark import SparkConf, SparkContext
from pyspark.sql.types import IntegerType, LongType, ArrayType, StringType, DoubleType
from pyspark.sql.functions import udf, explode, broadcast, count, lit, length, col
from pyspark.sql import DataFrame
from pyspark.sql.types import StructType

In [4]:
from pyspark.sql import SparkSession
from pyspark import SparkConf, SparkContext

## this works for PySpark v3.3.1 - only need to run this once
os.environ['PYSPARK_SUBMIT_ARGS'] = "--packages {aws_java},{aws_hadoop} pyspark-shell".\
   format(aws_java="com.amazonaws:aws-java-sdk-bundle:1.11.271",
          aws_hadoop="org.apache.hadoop:hadoop-aws:3.1.2")

os.environ["SPARK_HOME"] = "/home/ec2-user/mambaforge/envs/2023_06_15_BCdeconvolution_AS/lib/python3.7/site-packages/pyspark"
os.environ["SPARK_LOCAL_DIRS"] = "/temp"
os.environ["PYARROW_IGNORE_TIMEZONE"] = "1"

spark_conf = SparkConf()
spark_conf.set("spark.executor.instances", "2")
spark_conf.set("spark.executor.cores", "2")
spark_conf.set("spark.executor.memory", "16g")
spark_conf.set("spark.driver.memory", "64g")
spark_conf.set("spark.driver.maxResultSize", "32g")
spark_conf.set("spark.parquet.filterPushdown", "true")
spark_conf.set("spark.local.dir", "/temp")
spark_conf.getAll()

sc = SparkContext(conf=spark_conf)
sc.setLogLevel("ERROR")
spark = SparkSession(sc)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/06/17 00:14:57 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/06/17 00:14:57 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).
23/06/17 00:14:57 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/06/17 00:14:57 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


### CpG Map

In [5]:
cpg_map = pd.read_csv(CPG_MAP_PATH, usecols=['chr', 'start', 'end', 'cpg_index', 'cpg_index_hg38'], sep='\t')

In [6]:

%%time
ridxs = ~cpg_map['cpg_index_hg38'].isna()
hg19_hg38_map = dict(itertools.zip_longest(cpg_map[ridxs]['cpg_index'], cpg_map[ridxs]['cpg_index_hg38'].astype(int)))
hg38_hg19_map = dict(itertools.zip_longest(cpg_map[ridxs]['cpg_index_hg38'].astype(int), cpg_map[ridxs]['cpg_index']))


CPU times: user 15 s, sys: 4.87 s, total: 19.9 s
Wall time: 19.8 s


### Regions

In [8]:
region_df = pd.read_csv(REGION_PATH, sep='\t', usecols=range(0, 6), names=REGION_BED_COLS)

region_df['region_cpg_index_max'] -= 1
region_df.sort_values('region_cpg_index_min', inplace=True)
region_df['region_cpg_index_min_hg38'] = region_df['region_cpg_index_min'].map(hg19_hg38_map)
region_df['region_cpg_index_max_hg38'] = region_df['region_cpg_index_max'].map(hg19_hg38_map)

region_df.shape[0], region_df['region_id'].nunique()

(1658, 1658)

In [9]:
ridxs = ~region_df['region_cpg_index_min_hg38'].isna()
ridxs &= ~region_df['region_cpg_index_max_hg38'].isna()
region_df = region_df[ridxs].copy()
region_df.shape[0], region_df['region_id'].nunique()

(1658, 1658)

In [10]:
cg_count_hg19 = region_df['region_cpg_index_max']-region_df['region_cpg_index_min'] + 1
cg_count_hg38 = region_df['region_cpg_index_max_hg38']-region_df['region_cpg_index_min_hg38'] + 1
ridxs = (cg_count_hg19==cg_count_hg38)
ridxs &= (cg_count_hg19>=FILTER_CG_COUNT_REGION)
region_df = region_df[ridxs].copy()
region_df.shape[0], region_df['region_id'].nunique()

(1657, 1657)

In [11]:
region_df['region_cpg_index_min_hg38'] = region_df['region_cpg_index_min_hg38'].astype(int)
region_df['region_cpg_index_max_hg38'] = region_df['region_cpg_index_max_hg38'].astype(int)
region_df.dtypes

region_chr                   object
region_start                  int64
region_end                    int64
region_cpg_index_min          int64
region_cpg_index_max          int64
region_id                    object
region_cpg_index_min_hg38     int64
region_cpg_index_max_hg38     int64
dtype: object

### PAT PARQUET Files

In [12]:
PAT_COLS = [
    'sample_id', 'molecule_id', 'chr', 'number_molecules',
    'cpg_index_min', 'cpg_index_max', 'pat_string'
]

In [13]:
# pat_parquet_files = [spark.read.parquet(ifile).select(*PAT_COLS) for ifile in PARQUET_PATH_LIST_HG19]
# pat_hg19_ddf = functools.reduce(DataFrame.unionByName, pat_parquet_files)
# pat_hg19_ddf.printSchema()

In [14]:
pat_parquet_files = [spark.read.parquet(ifile).select(*PAT_COLS) for ifile in PARQUET_PATH_LIST_HG38]
pat_hg38_ddf = functools.reduce(DataFrame.unionByName, pat_parquet_files)
pat_hg38_ddf.printSchema()

root
 |-- sample_id: string (nullable = true)
 |-- molecule_id: string (nullable = true)
 |-- chr: string (nullable = true)
 |-- number_molecules: integer (nullable = true)
 |-- cpg_index_min: long (nullable = true)
 |-- cpg_index_max: long (nullable = true)
 |-- pat_string: string (nullable = true)



In [25]:
# # simple mixture
# pat_spikein = pat_hg38_ddf.filter(col('sample_id')=="ERS661063").sample(False, 0.1, seed=0)
# pat_sample = pat_hg38_ddf.filter(col('sample_id')=="ERS337607")

# pat_mix = pat_sample.union(pat_spikein)
#pat_hg38_ddf.select('sample_id').distinct().toPandas()

## Fragment Level Scoring

In [16]:
QUANTILES = [0.1, 0.25, 0.75, 0.9]
KMERS = [1, 3, 4]
RATES_LEQ = [0.25]
RATES_GEQ = [0.75]

RETURN_SCHEMA = StructType()\
    .add('sample_id', 'string')\
    .add('region_id', 'string')\
    .add('number_molecules', 'integer')\
    .add('meth_k1', 'integer')\
    .add('unmeth_k1', 'integer')\
    .add('total_k1', 'integer')\
    .add('meth_k3', 'integer')\
    .add('unmeth_k3', 'integer')\
    .add('total_k3', 'integer')\
    .add('meth_k4', 'integer')\
    .add('unmeth_k4', 'integer')\
    .add('total_k4', 'integer')\
    .add('frac_alpha_leq_25pct', 'float')\
    .add('frac_alpha_geq_75pct', 'float')

def compute_frag_scores(cpg_number_cutoff: int) -> pd.DataFrame:
    
    """
    Function that returns a function, used for reduce
    """
    
    def compute_frag_scores_inner(pat_df: pd.DataFrame) -> pd.DataFrame:
        
        data = pat_df.copy()
        data['offset_min'] = (data['region_cpg_index_min'] - data['cpg_index_min']).clip(lower=0)
        data['offset_max'] = np.minimum(
            data['region_cpg_index_max'] - data['cpg_index_min'], 
            data['cpg_index_max'] - data['cpg_index_min'])
        data['trimmed_pat'] = data.apply(lambda x: x['pat_string'][x['offset_min']:(x['offset_max']+1)], axis=1)
        #--- Filter molecules based on observed CpG loci
        observed_cpg_number = (data['trimmed_pat'].str.count('C')+data['trimmed_pat'].str.count('T'))
        ridxs = (observed_cpg_number>=cpg_number_cutoff)
        data = data[ridxs].copy()
        if (data.shape[0]>0):
            # Compute k-mer methylation states
            for k in KMERS:
                data['meth_k%i'%k] = data['trimmed_pat']\
                    .apply(lambda x: len(re.findall('[C]{%i}'%k, x, overlapped=True)))
                data['unmeth_k%i'%k] = data['trimmed_pat']\
                    .apply(lambda x: len(re.findall('[T]{%i}'%k, x, overlapped=True)))
                data['total_k%i'%k] = data['trimmed_pat']\
                    .apply(lambda x: len(re.findall('[TC]{%i}'%k, x, overlapped=True)))
            # Compute alpha distribution metrics
            data['alpha'] = data['meth_k1']/data['total_k1']
            for rate in RATES_LEQ:
                data['frac_alpha_leq_%ipct'%(100*rate)] = np.where(data['alpha']<=rate, 1, 0)
            for rate in RATES_GEQ:
                data['frac_alpha_geq_%ipct'%(100*rate)] = np.where(data['alpha']>=rate, 1, 0)
            # Expand entries that correspond to multiple molecules
            data['number_molecules'] = data['number_molecules'].apply(lambda x: list(range(x)))
            data = data.explode('number_molecules')
            data['number_molecules'] = 1
            # Aggregate metrics
            rv = data.groupby(['region_id', 'sample_id'])\
                [['meth_k1', 'unmeth_k1', 'total_k1',
                  'meth_k3', 'unmeth_k3', 'total_k3',
                  'meth_k4', 'unmeth_k4', 'total_k4',
                  'frac_alpha_leq_25pct', 'frac_alpha_geq_75pct', 'number_molecules']].sum()\
                .reset_index()
            rv['frac_alpha_leq_25pct'] = rv['frac_alpha_leq_25pct']/rv['number_molecules']
            rv['frac_alpha_geq_75pct'] = rv['frac_alpha_geq_75pct']/rv['number_molecules']
        else:
            rv = pd.DataFrame(columns=RETURN_SCHEMA.names)
                      
        
        return rv[RETURN_SCHEMA.names]

    return compute_frag_scores_inner


compute_frag_scores_udf = compute_frag_scores(cpg_number_cutoff=FILTER_CG_COUNT)


### Compute for HG19 Data

In [17]:
# %%time
# BATCH_SIZE = 20
# region_df['batch'] = (np.arange(region_df.shape[0])/BATCH_SIZE).astype(int)
# rv_scores = list()

# for batch, batch_region_df in region_df.groupby('batch'):
    
#     rv_ov = list()
#     print('---> Processing batch %i...' % batch)
    
#     for _, row in batch_region_df.iterrows():
        
#         # get PAT strings within each immune region
#         ov_ddf = pat_hg19_ddf.filter(col('cpg_index_min')<=row['region_cpg_index_max'])\
#             .filter(col('cpg_index_max')>=row['region_cpg_index_min'])\
#             .withColumn('region_id', lit(row['region_id']))\
#             .withColumn('region_cpg_index_min', lit(row['region_cpg_index_min']))\
#             .withColumn('region_cpg_index_max', lit(row['region_cpg_index_max']))
#         rv_ov.append(ov_ddf)
    
#     # for each region
#     scores_df = functools.reduce(DataFrame.union, rv_ov)\
#         .groupby('region_id')\
#         .applyInPandas(compute_frag_scores_udf, schema=RETURN_SCHEMA)\
#         .toPandas()
#     rv_scores.append(scores_df)

In [18]:
# scores_hg19_df = pd.concat(rv_scores)

### Compute for HG38 Data

In [19]:
%%time
BATCH_SIZE = 20
region_df['batch'] = (np.arange(region_df.shape[0])/BATCH_SIZE).astype(int)
rv_scores = list()
for batch, batch_region_df in region_df.groupby('batch'):
    rv_ov = list()
    print('---> Processing batch %i...' % batch)
    for _, row in batch_region_df.iterrows():
        ov_ddf = pat_hg38_ddf.filter(col('cpg_index_min')<=row['region_cpg_index_max_hg38'])\
            .filter(col('cpg_index_max')>=row['region_cpg_index_min_hg38'])\
            .withColumn('region_id', lit(row['region_id']))\
            .withColumn('region_cpg_index_min', lit(row['region_cpg_index_min_hg38']))\
            .withColumn('region_cpg_index_max', lit(row['region_cpg_index_max_hg38']))
        rv_ov.append(ov_ddf)
    scores_df = functools.reduce(DataFrame.union, rv_ov)\
        .groupby('region_id')\
        .applyInPandas(compute_frag_scores_udf, schema=RETURN_SCHEMA)\
        .toPandas()
    rv_scores.append(scores_df)

---> Processing batch 0...


---> Processing batch 1...


---> Processing batch 2...


---> Processing batch 3...


---> Processing batch 6...


...

---> Processing batch 79...


---> Processing batch 80...


---> Processing batch 81...


---> Processing batch 82...


CPU times: user 15.3 s, sys: 2.39 s, total: 17.7 s
Wall time: 1h 3min 30s


In [20]:
scores_hg38_df = pd.concat(rv_scores)

## Write Out

In [21]:
#scores_df = pd.concat([scores_hg19_df, scores_hg38_df])
scores_df = scores_hg38_df
scores_df.shape[0], scores_df['region_id'].nunique(), scores_df['sample_id'].nunique()

(206303, 1648, 127)

In [22]:
%%time
scores_df.to_csv(('output/analysis/gh-aselewa/projects/2023_06_15_BCdeconvolution_AS/meth_summaries_cg_count_geq_{k}_{regions}.tsv.gz').format(regions=REGIONS, k=FILTER_CG_COUNT),
                 sep='\t', 
                 index=False)

CPU times: user 3.13 s, sys: 118 µs, total: 3.13 s
Wall time: 3.13 s
